In [4]:
pip install geopandas



   ---------------------------------------- 0.0/19.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/19.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/19.2 MB ? eta -:--:--
    --------------------------------------- 0.3/19.2 MB ? eta -:--:--
    --------------------------------------- 0.3/19.2 MB ? eta -:--:--
   - -------------------------------------- 0.5/19.2 MB 578.7 kB/s eta 0:00:33
   - -------------------------------------- 0.5/19.2 MB 578.7 kB/s eta 0:00:33
   - -------------------------------------- 0.5/19.2 MB 578.7 kB/s eta 0:00:33
   - -------------------------------------- 0.8/19.2 MB 550.1 kB/s eta 0:00:34
   - -------------------------------------- 0.8/19.2 MB 550.1 kB/s eta 0:00:34
   -- ------------------------------------- 1.0/19.2 MB 565.8 kB/s eta 0:00:33
   -- ------------------------------------- 1.3/19.2 MB 610.0 kB/s eta 0:00:30
   -- ------------------------------------- 1.3/19.2 MB 610.0 kB/s eta 0:00:30
   --- ----------

In [8]:
!pip install folium




In [1]:
import geopandas as gpd
import pandas as pd
import folium
from sklearn.cluster import DBSCAN
import numpy as np

df=pd.read_csv('../data/cleaned/cleaned_missing_data.csv')

# Sample lat/lon dict (expand with real coords; e.g., Andhra Pradesh: 15.9129, 79.7400)
state_coords = {
    'Andhra Pradesh': (15.9129, 79.7400),
    'Arunachal Pradesh': (28.2180, 94.7278),
    'Assam': (26.2006, 92.9376),
    'Bihar': (25.0961, 85.3131),
    'Chhattisgarh': (21.2787, 81.8661),
    'Goa': (15.2993, 74.1240),
    'Gujarat': (22.2587, 71.1924),
    'Haryana': (29.0588, 76.0856),
    'Himachal Pradesh': (31.1048, 77.1734),
    'Jharkhand': (23.6102, 85.2799),
    'Karnataka': (15.3173, 75.7139),
    'Kerala': (10.8505, 76.2711),
    'Madhya Pradesh': (22.9734, 78.6569),
    'Maharashtra': (19.7515, 75.7139),
    'Manipur': (24.6637, 93.9063),
    'Meghalaya': (25.4670, 91.3662),
    'Mizoram': (23.1645, 92.9376),
    'Nagaland': (26.1584, 94.5624),
    'Odisha': (20.9517, 85.0985),
    'Punjab': (31.1471, 75.3412),
    'Rajasthan': (27.0238, 74.2179),
    'Sikkim': (27.5330, 88.5122),
    'Tamil Nadu': (11.1271, 78.6569),
    'Telangana': (18.1124, 79.0193),
    'Tripura': (23.9408, 91.9882),
    'Uttar Pradesh': (26.8467, 80.9462),
    'Uttarakhand': (30.0668, 79.0193),
    'West Bengal': (22.9868, 87.8550),
    
    # Union Territories
    'Andaman and Nicobar Islands': (11.7401, 92.6586),
    'Chandigarh': (30.7333, 76.7794),
    'Dadra and Nagar Haveli and Daman and Diu': (20.1809, 73.0169),
    'Delhi': (28.7041, 77.1025),
    'Jammu and Kashmir': (33.7782, 76.5762),
    'Ladakh': (34.2268, 77.5619),
    'Lakshadweep': (10.5667, 72.6417),
    'Puducherry': (11.9416, 79.8083)
}


df['lat'] = df['state_name'].map(lambda x: state_coords.get(x, (np.nan, np.nan))[0])
df['lon'] = df['state_name'].map(lambda x: state_coords.get(x, (np.nan, np.nan))[1])

gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat))

# Heatmap
m = folium.Map(location=[20.5937, 78.9629], zoom_start=5)  # India center
from folium.plugins import HeatMap
HeatMap(data=gdf[['lat', 'lon', 'total_missing']].dropna(), radius=15).add_to(m)
m.save('../visual/missing_heatmap.html')


# Clustering hotspots
coords_df = gdf.dropna(subset=['lat', 'lon']).copy()
coords = coords_df[['lat', 'lon']].values

db = DBSCAN(eps=5, min_samples=3).fit(coords)  # eps is in degrees
coords_df['cluster'] = db.labels_

# Merge cluster labels back to original gdf
gdf = gdf.merge(coords_df[['cluster']], left_index=True, right_index=True, how='left')

# Replace NaN with descriptive label
gdf['cluster'] = gdf['cluster'].fillna('Unknown location')

print(gdf['cluster'].value_counts())


cluster
0.0                 3163
1.0                  173
Unknown location     142
Name: count, dtype: int64
